In [2]:
# Create LogStatus table from ELEMENT DB using an ODBC connection through SQLAlchemy
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
server = 'DOH-LAB-04\ELEMENT,1493'
database = 'LTDB'
engine = create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=SQL+Server', use_setinputsizes=False)
work = '23I2701'
query = (
         "select distinct s.SampleInfo1, a.Wrk, a.Sample, n.Batch, a.Analysis, a.Analyte, a.nResult, a.nMDL, a.nMRL, l.LR, a.FinalUnits, a.AnalyteNotes, q.Qualifier, q.TextBody, s.SampleReceived, a.Analyzed "
         "from LTDB.dbo.WRKSAMPLE s "
         "inner join LTDB.dbo.REPSAMPLEANALYTE a "
         "on s.Wrk=a.Wrk and s.Sample=a.Sample "
         "inner join LTDB.dbo.REPSAMPLEANALYSIS n "
         "on a.Wrk=n.Wrk and a.Sample=n.Sample "
         "inner join LTDB.dbo.ANALYSISLIMITSET l "
         "on a.Analysis=l.Analysis and a.Analyte=l.Analyte "
         "left join LTDB.dbo.REPQUALIFIER q "
         "on a.Wrk=q.Wrk and a.Sample=q.Sample and a.Analyte=q.Analyte "
         "where a.Wrk='" +work+ "' order by Wrk, Sample, Analyte"
        )
df = pd.read_sql(query, engine)
from datetime import datetime
today = datetime.today().strftime('%m%d%y')

# Subset ELEMENT data with limited variables to further explore
df2 = df[['Wrk', 'Sample', 'Batch', 'Analysis', 'Analyte', 'nResult']]
Unique = df2.nunique()
Unique2 = Unique.to_frame()
import dataframe_image as dfi
df_styled = Unique2.style.background_gradient()
#dfi.export(df_styled,'V:\\Biomonitoring Grant-CDC 2019-2024\\University of Arizona\\Firefighter Full Project\\'+work+'-Unique Count-'+datetime.today().strftime('%m%d%y')+'.png')

# Interactive image on browser with hovering individual data
import plotly
import plotly.io as pio
import plotly.express as px
pio.renderers.default='browser'
fig = px.box(df2, x="nResult", y="Analyte", hover_data=["Sample"], points='all', title='Interactive Box Plot for Work Order of '+work+' by Analyte')
fig.update_yaxes(categoryorder='median ascending')
fig.show()